# GEEMAP EU Bounds

The EU is not available as a region of Interest in geemap. We have here a script which concatenates a list of European Countries, which are part of the EU, and returns the bounds of continental European countries.


We can futher see that the obtained results corresponds to the Map of LUCAS soil samples.

In [6]:
%%capture
!pip install geemap

In [7]:
%%capture
!pip install pycountry

In [8]:
import numpy as np
import os

In [5]:
import geemap
import ee
ee.Authenticate()
ee.Initialize(project='ee-encodersmap1')


In [9]:
import numpy as np

# List of EU countries in the LUCAS SOIL dataset
eu_countries = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Denmark',
    'Estonia', 'France', 'Germany', 'Greece', 'United Kingdom', 'Hungary',
    'Czechia', 'Ireland', 'Italy', 'Latvia', 'Finland', 'Lithuania',
    'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania',
    'Slovakia', 'Slovenia', 'Spain', 'Sweden'
]

# Land areas corresponding to the countries
eu_countries_landArea = np.array([83871, 30688, 110994, 56594, 9251, 42952,
                                  45339, 551695, 357588, 131957, 243610, 93026,
                                  78867, 84421, 302073, 64589, 338462, 65300,
                                  2586, 316, 41850, 322575, 92152, 238397,
                                  49035, 20273, 505990, 528447])

# Normalizing the land area
eu_countries_landAreaNormalized = np.round(eu_countries_landArea / eu_countries_landArea.sum() * 1000000)

# Create the dictionary
eu_countries_dict = {country: {'land_area': land_area, 'normalized_land_area': norm_land_area}
                     for country, land_area, norm_land_area
                     in zip(eu_countries, eu_countries_landArea, eu_countries_landAreaNormalized)}

eu_countries_dict


{'Austria': {'land_area': 83871, 'normalized_land_area': 18667.0},
 'Belgium': {'land_area': 30688, 'normalized_land_area': 6830.0},
 'Bulgaria': {'land_area': 110994, 'normalized_land_area': 24704.0},
 'Croatia': {'land_area': 56594, 'normalized_land_area': 12596.0},
 'Cyprus': {'land_area': 9251, 'normalized_land_area': 2059.0},
 'Denmark': {'land_area': 42952, 'normalized_land_area': 9560.0},
 'Estonia': {'land_area': 45339, 'normalized_land_area': 10091.0},
 'France': {'land_area': 551695, 'normalized_land_area': 122793.0},
 'Germany': {'land_area': 357588, 'normalized_land_area': 79590.0},
 'Greece': {'land_area': 131957, 'normalized_land_area': 29370.0},
 'United Kingdom': {'land_area': 243610, 'normalized_land_area': 54221.0},
 'Hungary': {'land_area': 93026, 'normalized_land_area': 20705.0},
 'Czechia': {'land_area': 78867, 'normalized_land_area': 17554.0},
 'Ireland': {'land_area': 84421, 'normalized_land_area': 18790.0},
 'Italy': {'land_area': 302073, 'normalized_land_area':

In [10]:
eu_countries_landAreaNormalized

array([1.86670e+04, 6.83000e+03, 2.47040e+04, 1.25960e+04, 2.05900e+03,
       9.56000e+03, 1.00910e+04, 1.22793e+05, 7.95900e+04, 2.93700e+04,
       5.42210e+04, 2.07050e+04, 1.75540e+04, 1.87900e+04, 6.72330e+04,
       1.43760e+04, 7.53330e+04, 1.45340e+04, 5.76000e+02, 7.00000e+01,
       9.31500e+03, 7.17970e+04, 2.05110e+04, 5.30610e+04, 1.09140e+04,
       4.51200e+03, 1.12620e+05, 1.17618e+05])

In [11]:
print(eu_countries)


['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Denmark', 'Estonia', 'France', 'Germany', 'Greece', 'United Kingdom', 'Hungary', 'Czechia', 'Ireland', 'Italy', 'Latvia', 'Finland', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']


In [12]:

# Initialize an empty bounds geometry
bounds = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
        .filter(ee.Filter.eq('country_na', eu_countries[0])) \
        .geometry()


print(bounds)
# Iterate over the countries and union their bounds
eu_countries = eu_countries[1:]
for country in eu_countries:
  print(country)
    # Get the bounds of the country
  country_bounds = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
        .filter(ee.Filter.eq('country_na', country)) \
        .geometry()
  bounds = bounds.union(country_bounds)



    # Union the country bounds with the existing bound


# intersects_bounds = bounds.intersects(roi2)


roi3 = ee.Geometry.Polygon(bounds) # ee.Geometry.Polygon(intersects_bounds) #roi2.geometry()
# Load the AVHRR LAI/FAPAR image collection

#collection = ee.ImageCollection(collection_name).filterDate(start_date, end_date).select(bands_name)



# Filter the image collection based on the ROI
#filtered_collection = collection.filterBounds(roi3)

# Get the first image from the filtered collection
# image = filtered_collection.first()

# Create a geemap map
Map = geemap.Map()

# Add the clipped image layer to the map
# Map.addLayer(image.clip(roi2), {'bands': ['GPP'], 'min': 0, 'max': 100}, 'AVHRR LAI')
# Create a gray layer with opacity 0.5
gray_layer = ee.Image().byte().paint(roi3, 128, 30)

# Define visualization parameters for the gray layer
vis_params = {
    'palette': '808080',   # Gray color
    'opacity': 1.0         # Opacity of 0.5
}

# Map.addLayer(roi3.bounds(), {'color': 'red'}, 'Berlin')

# Add the gray layer to the map
# Map.addLayer(gray_layer, vis_params, 'Gray Layer')
Map.addLayer(bounds, {}, 'Intersected Bounds')

# Display the map
Map

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.filter",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.loadTable",
                "arguments": {
                  "tableId": {
                    "constantValue": "USDOS/LSIB_SIMPLE/2017"
                  }
                }
              }
            },
            "filter": {
              "functionInvocationValue": {
                "functionName": "Filter.equals",
                "arguments": {
                  "leftField": {
                    "constantValue": "country_na"
                  },
                  "rightValue": {
                    "constantValue": "Austria"
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

We have below on the MAP, the LUCAS Dataset, overlapping with the bounds we have crearte here.

This may be used for future plotting.

For Donwloading using those Bounds are slowing the downloading much. Using In Situ coordinates is much better.

In [13]:
type(bounds)

ee.geometry.Geometry

In [14]:
import numpy as np

In [15]:

# Authenticate Earth Engine
# Make sure to authenticate your Google Earth Engine account before running this code

# Set the list of EU countries

# Define the parent directory where the folders will be created
parent_directory = '/content/'



# Iterate over the list of countries
for i in range(len(eu_countries)):
    country = eu_countries[i]
    land_area = eu_countries_dict[country]['normalized_land_area']

    folder_name = f'{country}'

    # Create a folder for the country
    folder_path = os.path.join(parent_directory, folder_name)
    os.makedirs(folder_path, exist_ok=True)

    # Generate random locations
    bounds2 = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
        .filter(ee.Filter.eq('country_na', country)) \
        .geometry()

    # Define the number of random points to generate
    cumulative_sum = 0
    count = 0
    while cumulative_sum < land_area:
      count +=1
      num_points = 5000
      print(cumulative_sum)
      cumulative_sum += num_points
    # Generate random points within the bounds of the geometry
      random_points = ee.FeatureCollection.randomPoints(bounds2, num_points)

    # Assuming you have a list of dictionaries
      list_of_dicts = random_points.getInfo()['features']

    # Initialize an empty list to store the coordinates
      coordinates = []

    # Iterate over the list of dictionaries
      for item in list_of_dicts:
        # Access the nested 'geometry' dictionary
        geometry_dict = item['geometry']
        # Access the 'coordinates' from the nested 'geometry' dictionary
        coords = geometry_dict['coordinates']
        # Append the coordinates to the list
        coordinates.append(coords)

    # Print the list of coordinates

    # Convert the 'coordinates' list to a NumPy array
      array_coords = np.array(coordinates)

    # Save the NumPy array to a .npy file
      np.save(os.path.join(folder_path, f'coordinates_{country+str(count)}.npy'), array_coords)


0
5000
0
5000
10000
15000
20000
0
5000
10000
0
0
5000
0
5000
10000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
0
5000
10000
15000
20000
25000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
0
5000
10000
15000
20000
0
5000
10000
15000
0
5000
10000
15000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
0
5000
10000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
0
5000
10000
0
0
0
5000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
0
5000
10000
15000
20000
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
0
5000
10000
0
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
0
5000
1

In [16]:
import os
import numpy as np
from google.colab import files

# Set the list of EU countries

# Define the parent directory where the folders are located
parent_directory = '/content/'

# Initialize an empty list to store the concatenated arrays
concatenated_array = []

# Iterate over the list of countries
for country in eu_countries:
    folder_path = os.path.join(parent_directory, country)

    # Check if the folder exists
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        # Iterate over the files in the folder
        for filename in os.listdir(folder_path):
            if filename.endswith('.npy'):
                file_path = os.path.join(folder_path, filename)
                array = np.load(file_path)
                concatenated_array.append(array)

    # Concatenate the arrays
    result_array = np.concatenate(concatenated_array, axis=0)

    # Define the file path and name for the result array
    file_name = f"coordinates{country}.npy"
    file_path = os.path.join(parent_directory, file_name)

# Save the result array as a .npy file
    np.save(file_path, result_array)

# Download the .npy file
   # files.download(file_path)


In [17]:
import numpy as np

# Specify the file path of the .npy file
file_path = '/content/United Kingdom/coordinates_United Kingdom6.npy'

# Load the .npy file
loaded_array = np.load(file_path)

# View the shape of the loaded array
print("Array shape:", loaded_array.shape)


Array shape: (5000, 2)


In [18]:
loaded_array

array([[-2.16676608, 52.8386022 ],
       [-2.15863503, 57.02213954],
       [-7.25875588, 54.79108851],
       ...,
       [ 0.82720882, 52.49776372],
       [ 0.79319787, 52.31923131],
       [-3.54828682, 52.51643346]])

In [19]:
import os
import shutil
from zipfile import ZipFile
from google.colab import files

# Set the list of EU countries

# Define the parent directory where the folders are located
parent_directory = '/content/'

# Create a temporary directory for storing the zipped folders
temp_directory = '/content/'
os.makedirs(temp_directory, exist_ok=True)

# Iterate over the list of countries
for country in eu_countries:
    folder_path = os.path.join(parent_directory, country)

    # Check if the folder exists
    if os.path.exists(folder_path) and os.path.isdir(folder_path):
        # Zip the folder and save it to the temporary directory
        zip_path = os.path.join(temp_directory, f'{country}')
        shutil.make_archive(zip_path, 'zip', folder_path)

        # Download the zipped folder
       # files.download(zip_path)

        # Remove the zipped folder from the temporary directory


In [20]:
import shutil
import os
# Step 2: Create the '1MilCoordinatesMLTRANS' folder inside /content/
target_folder = '/content/1MilCoordinatesMLTRANS'
os.makedirs(target_folder, exist_ok=True)

# Step 3: Move only folders from /content/ (excluding the target folder itself) into '1MilCoordinatesMLTRANS'
for item in os.listdir('/content/'):
    item_path = os.path.join('/content/', item)

    # Check if the item is a folder and not the target folder itself
    if os.path.isdir(item_path) and item_path != target_folder:
        shutil.move(item_path, target_folder)

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


# Step 4: Zip the '1MilCoordinatesMLTRANS' folder
zip_file = '/content/1MilCoordinatesMLTRANS_Redux.zip'
shutil.make_archive(zip_file.replace('.zip', ''), 'zip', target_folder)

# Step 5: Move the zip file to the destination in Google Drive
destination_drive_directory = '/content/drive/MyDrive/Colab Notebooks/VaesSOC/Data_Files_Redux/CoordinatesPoints/'
shutil.move(zip_file, destination_drive_directory)

print(f'Zip file moved to {destination_drive_directory}')

Mounted at /content/drive
Zip file moved to /content/drive/MyDrive/Colab Notebooks/VaesSOC/Data_Files_Redux/


In [21]:
# Define the number of random points to generate
num_points = 1

# Generate random points within the bounds of the geometry
random_points = ee.FeatureCollection.randomPoints(bounds, num_points)

# Convert the random points to a geemap feature collection

# Visualize the random points
Map = geemap.Map(center=[40.5, -73.5], zoom=8)
Map.addLayer(random_points, {}, "Random Points")
Map

Map(center=[40.5, -73.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [22]:
import numpy as np
# Initialize a NumPy matrix to store the coordinates
coordinates = np.zeros((2, num_points))

# Iterate over the random points feature collection and extract the coordinates
for i, feature in enumerate(random_points.getInfo()['features']):
    geometry = ee.Feature(feature).geometry()
    lon, lat = geometry.getInfo()['coordinates']
    coordinates[:, i] = [lon, lat]

print(coordinates)

[[15.17554594]
 [51.35895703]]


In [23]:
# Create an instance of geemap
Map = geemap.Map()

# Load the feature collection
fc = ee.FeatureCollection("JRC/LUCAS_HARMO/THLOC/V1")

# Display all the bands of the feature collection
Map.addLayer(fc, {}, 'Feature Collection')
features10000 = geemap.fishnet(bounds, rows=100, cols=100, delta=1)
Map.addLayer(features10000, {}, 'Fishnet 2')
# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…